In [19]:

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

if torch.cuda.is_available():
    print("Yeah we have a GPU!")
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

import torchvision
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import itertools

from gdeep.decision_boundary import GradientFlowDecisionBoundaryCalculator

# Train CNN-Autoencoder

In [3]:
# https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/conv_autoencoder.py


if not os.path.exists('/home/reinauer/EPFL/giotto-deep/examples/mlp_img'):
    os.mkdir('/home/reinauer/EPFL/giotto-deep/examples/mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Linear(64, 12),
            nn.ReLU(True),
            nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model_auto = autoencoder().to(dev)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model_auto.parameters(), lr=learning_rate, weight_decay=1e-5)

In [ ]:
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).to(dev)
        # ===================forward=====================
        output = model_auto(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, '/home/reinauer/EPFL/giotto-deep/examples/mlp_img/image_{}.png'.format(epoch))

torch.save(model_auto, '/home/reinauer/EPFL/giotto-deep/examples/sim_autoencoder.pth')

In [4]:
model_auto = torch.load('/home/reinauer/EPFL/giotto-deep/examples/sim_autoencoder.pth')
model_auto.eval()

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=12, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=12, out_features=3, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=128, out_features=784, bias=True)
    (7): Tanh()
  )
)

In [112]:
embedding_df = pd.DataFrame(columns=list('xyzc'))

for x, y in itertools.islice(dataloader, 20):
    embedded_batch = model_auto.encoder(x.view(x.size(0), -1)).detach().numpy()
    embedded_batch_df = pd.DataFrame(
        np.concatenate((embedded_batch, y.unsqueeze(1).numpy()), 1),
        columns=list('xyzc'))
    embedding_df = embedding_df.append(
        embedded_batch_df,
        ignore_index=True)

In [113]:
fig = px.scatter_3d(embedding_df, x='x', y='y', z='z',
              color='c')
fig.show()

In [5]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


transform=transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.1307,), (0.3081,))
    ])

batch_size = 64
test_batch_size = 64
train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

dataset1 = datasets.MNIST('./data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('./data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [98]:
seed = 42
torch.manual_seed(seed)
batch_size = 64
test_batch_size = 64
lr = 0.01
gamma = 0.7
epochs = 14
log_interval = 10

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

transform=transforms.Compose([
    transforms.ToTensor()
    ])


model = Net().to(dev)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(log_interval, model, dev, train_loader, optimizer, epoch)
    test(model, dev, test_loader)
    scheduler.step()

 (86%)]	Loss: 0.282271
Train Epoch: 10 [52480/60000 (87%)]	Loss: 0.174014
Train Epoch: 10 [53120/60000 (88%)]	Loss: 0.404752
Train Epoch: 10 [53760/60000 (90%)]	Loss: 0.179897
Train Epoch: 10 [54400/60000 (91%)]	Loss: 0.254622
Train Epoch: 10 [55040/60000 (92%)]	Loss: 0.259375
Train Epoch: 10 [55680/60000 (93%)]	Loss: 0.475128
Train Epoch: 10 [56320/60000 (94%)]	Loss: 0.253091
Train Epoch: 10 [56960/60000 (95%)]	Loss: 0.364380
Train Epoch: 10 [57600/60000 (96%)]	Loss: 0.345194
Train Epoch: 10 [58240/60000 (97%)]	Loss: 0.119694
Train Epoch: 10 [58880/60000 (98%)]	Loss: 0.105647
Train Epoch: 10 [59520/60000 (99%)]	Loss: 0.119685

Test set: Average loss: 0.2391, Accuracy: 9282/10000 (93%)

Train Epoch: 11 [0/60000 (0%)]	Loss: 0.317941
Train Epoch: 11 [640/60000 (1%)]	Loss: 0.308584
Train Epoch: 11 [1280/60000 (2%)]	Loss: 0.477598
Train Epoch: 11 [1920/60000 (3%)]	Loss: 0.251033
Train Epoch: 11 [2560/60000 (4%)]	Loss: 0.282096
Train Epoch: 11 [3200/60000 (5%)]	Loss: 0.312106
Train Epoch: 1

In [6]:
#torch.save(model, '/home/reinauer/EPFL/giotto-deep/examples/cnn_minst_10.pth')
model = torch.load('/home/reinauer/EPFL/giotto-deep/examples/cnn_minst_10.pth')
model.eval()

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [15]:
class LambdaLayer(nn.Module):
    def __init__(self, lam):
        super().__init__()
        self.lam = lam
    def forward(self, x):
        return self.lam(x)

class DBMulticlass(nn.Module):
    def forward(self, x):
        y = torch.topk(x, 2).values
        return (y[:, 0] - y[:, 1])**2

embedding_model = nn.Sequential(model_auto.decoder,
                    LambdaLayer(lambda x: x.reshape(-1, 1, 28, 28)),
                    model,
                    DBMulticlass())

In [17]:
n_delta = 20
t = np.linspace(-10.5, 10.5, n_delta)
x, y, z = np.meshgrid(t,t,t)
xyz = torch.from_numpy(np.stack((x, y, z), axis=-1)).reshape(-1, 3)
xyz = xyz.type(torch.float32)
values = embedding_model(xyz).reshape(n_delta, n_delta, n_delta,-1).detach()
   
trace_1 = go.Isosurface(
    x = x.flatten(),
    y = y.flatten(),
    z = z.flatten(),
    value=values.flatten(),
    isomin=0.0,
    isomax=0.1)

fig = go.FigureWidget([trace_1])
fig.show()


In [22]:
n_steps = 1000
precision = 0.1

initial_points_batch = 10. * torch.rand((600, 3)) - 5.
initial_points_batch.to(dev, non_blocking=True)
g = GradientFlowDecisionBoundaryCalculator(
            model=embedding_model,
            initial_points=initial_points_batch,
            optimizer=lambda params: torch.optim.Adam(params)
)
g.step(n_steps)

db_sample = g.get_filtered_decision_boundary(precision).detach().numpy()


In [ ]:
print(db_samples.)

In [28]:
db_sample_df = pd.DataFrame(db_sample, columns = ["x"+str(i) for i in range(3)])

fig = px.scatter_3d(db_sample_df, x='x0', y='x1', z='x2')
fig.show()